In [3]:
!pip install torch==1.6.0
!pip install opencv-python
!pip install torchvision==0.2.2
!pip install albumentations
!pip install tensorflow
!pip install pytorch-lightning

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 748.8 MB 16 kB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 49.5 MB 29.4 MB/s 
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 64 kB 5.6 MB/s 
     |████████████████████████████████| 52 kB 3.7 MB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user i

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import typing as tp
import yaml
import random
import os
import sys
import soundfile as sf
import librosa
import cv2
import matplotlib.pyplot as plt
import time
import glob
from tqdm import tqdm

import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
# import resnest.torch as resnest_torch

from torchvision import models

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
# from resnet import ResNet, Bottleneck

from albumentations.core.transforms_interface import DualTransform, BasicTransform
import albumentations as albu

from sklearn.model_selection import StratifiedKFold

pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

## util

In [2]:
config_set = {
    'dataset': {
          'name': 'SpectrogramDataset',
          'params': {
            'img_size': 224, 
            'melspectrogram_parameters': {
              'n_mels': 128, 
              'fmin': 50, 
              'fmax': 15000, 
            }
      }
    },
    'loader': {
      'train': {
        'batch_size': 6,
        'shuffle': True,
        'num_workers': 2,
        'pin_memory': True,
        'drop_last': True,
      },
      'valid': {
        'batch_size': 2,
        'shuffle': False,
        'num_workers': 2,
        'pin_memory': True,
        'drop_last': True,
      }
    }
}
SEED=100
PERIOD = 5
SPECIES_NUM = 24
EPOCH = 50
HOP_LEN = 512
SR = 48000

In [3]:
config = config_set

In [4]:
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
set_seed(SEED)

In [5]:
INPUT_ROOT = Path("/home/knikaido/work/Rainforest-Connection/data")
RAW_DATA = INPUT_ROOT / "rfcx-species-audio-detection"
TRAIN_AUDIO_DIR = RAW_DATA / "train"
# TRAIN_RESAMPLED_AUDIO_DIRS = [
#   INPUT_ROOT / "birdsong-resampled-train-audio-{:0>2}".format(i)  for i in range(5)
# ]
TEST_AUDIO_DIR = RAW_DATA / "test"
OUTPUT_DIR = './output/'

In [6]:
OUTPUT_DIR1 = '../175/output/'
OUTPUT_DIR2 = '../153/output/'
OUTPUT_DIR3 = '../178/output/'
OUTPUT_DIR4 = '../182/output/'
OUTPUT_DIR5 = '../184/output/'
OUTPUT_DIR6 = '../185/output/'

In [7]:
pred_pathes = sorted(glob.glob(OUTPUT_DIR1 + '*[0-9].csv'))
pred_pathes.extend(sorted(glob.glob(OUTPUT_DIR2 + '*[0-9].csv')))
pred_pathes.extend(sorted(glob.glob(OUTPUT_DIR3 + '*[0-9].csv')))
pred_pathes.extend(sorted(glob.glob(OUTPUT_DIR4 + '*[0-9].csv')))
pred_pathes.extend(sorted(glob.glob(OUTPUT_DIR5 + '*[0-9].csv')))
pred_pathes.extend(sorted(glob.glob(OUTPUT_DIR6 + '*[0-9].csv')))
pred_pathes

['../175/output/175_sub0.csv',
 '../175/output/175_sub1.csv',
 '../175/output/175_sub2.csv',
 '../175/output/175_sub3.csv',
 '../175/output/175_sub4.csv',
 '../153/output/153_sub0.csv',
 '../153/output/153_sub1.csv',
 '../153/output/153_sub2.csv',
 '../153/output/153_sub3.csv',
 '../153/output/153_sub4.csv',
 '../178/output/178_sub0.csv',
 '../178/output/178_sub1.csv',
 '../178/output/178_sub2.csv',
 '../178/output/178_sub3.csv',
 '../182/output/153_sub0.csv',
 '../182/output/153_sub1.csv',
 '../182/output/153_sub2.csv',
 '../182/output/153_sub3.csv',
 '../184/output/184_sub0.csv',
 '../184/output/184_sub1.csv',
 '../184/output/184_sub2.csv',
 '../184/output/184_sub3.csv',
 '../184/output/184_sub4.csv',
 '../185/output/185_sub0.csv',
 '../185/output/185_sub1.csv',
 '../185/output/185_sub2.csv',
 '../185/output/185_sub3.csv']

In [8]:
df_pred = np.zeros([1992, 24])
for path in pred_pathes:
    df_pred += pd.read_csv(path).iloc[:, 1:].values

In [9]:
df_pred /= len(pred_pathes)

In [10]:
sub = pd.read_csv(str(RAW_DATA / 'sample_submission.csv'))
sub.loc[:, 's0':'s23'] = df_pred

In [11]:
sub.to_csv(OUTPUT_DIR + 'ansamble1_1.csv', index=False)

In [12]:
sub

,recording_id,s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,s22,s23
0,000316da7,0.039698,0.000254,0.003515,0.175306,0.000526,0.023245,0.000633,0.001518,0.001212,0.001947,0.000212,0.000423,0.202428,0.000644,0.000435,0.000614,0.005861,0.000886,0.071120,0.000587,0.000912,0.007751,0.000492,0.002756
1,003bc2cb2,0.000259,0.002984,0.003678,0.070388,0.000374,0.001786,0.000296,0.006868,0.000345,0.000198,0.001078,0.000189,0.003983,0.002630,0.010635,0.001205,0.699236,0.010915,0.001819,0.000252,0.000727,0.002398,0.000756,0.000909
2,0061c037e,0.001621,0.000445,0.005640,0.040893,0.001249,0.010379,0.000709,0.021139,0.000160,0.001572,0.000648,0.001650,0.005670,0.000400,0.000185,0.014859,0.001497,0.007097,0.000544,0.000658,0.001605,0.000388,0.004295,0.003616
3,010eb14d3,0.789543,0.000363,0.000088,0.000864,0.000108,0.000161,0.000143,0.000097,0.174883,0.000180,0.000122,0.000043,0.000280,0.000177,0.001010,0.000076,0.000156,0.000130,0.012546,0.000279,0.000092,0.000205,0.000153,0.000166
4,011318064,0.000823,0.000783,0.002586,0.081526,0.000553,0.006262,0.002553,0.003049,0.000480,0.000670,0.000505,0.004356,0.002068,0.001099,0.495483,0.127205,0.002447,0.001703,0.029530,0.000874,0.001207,0.002097,0.000571,0.000847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1987,ff68f3ac3,0.003900,0.007815,0.002140,0.071836,0.004898,0.183777,0.000744,0.002641,0.000545,0.000658,0.000513,0.001107,0.063236,0.018975,0.000691,0.235396,0.000437,0.000573,0.009022,0.001069,0.001286,0.001033,0.000732,0.239239
1988,ff973e852,0.000465,0.000307,0.001066,0.027066,0.000179,0.001666,0.002872,0.479175,0.000285,0.030358,0.001445,0.010626,0.000744,0.000198,0.003371,0.067343,0.001411,0.138284,0.000391,0.000911,0.005795,0.000168,0.010059,0.003869
1989,ffa5cf6d6,0.001047,0.005491,0.020735,0.286707,0.000389,0.014061,0.000781,0.006206,0.000203,0.019727,0.000671,0.001128,0.020046,0.001599,0.000587,0.287422,0.002596,0.064597,0.000462,0.000954,0.002284,0.000562,0.001731,0.004297
1990,ffa88cbb8,0.004662,0.010999,0.008579,0.400463,0.000632,0.004692,0.000472,0.195379,0.000391,0.017303,0.000577,0.000599,0.053677,0.006988,0.001700,0.010639,0.090970,0.004771,0.002065,0.000403,0.002236,0.006806,0.000793,0.005966


In [6]:
train_gby = pd.read_pickle(RAW_DATA / "train_gby_mel.pkl")
train_gby.head()

,recording_id,species_id,songtype_id,t_min,f_min,t_max,f_max,name
0,003bec244,[14],[1],[44.544],[2531.25],[45.1307],[5531.25],/home/knikaido/work/Rainforest-Connection/Git/...
1,006ab765f,[23],[1],[39.9615],[7235.16],[46.0452],[11283.4],/home/knikaido/work/Rainforest-Connection/Git/...
2,007f87ba2,[12],[1],[39.135999999999996],[562.5],[42.272],[3281.25],/home/knikaido/work/Rainforest-Connection/Git/...
3,0099c367b,[17],[4],[51.4206],[1464.26],[55.1996],[4565.04],/home/knikaido/work/Rainforest-Connection/Git/...
4,009b760e6,[10],[1],[50.0854],[947.461],[52.5293],[10852.7],/home/knikaido/work/Rainforest-Connection/Git/...


In [7]:
def mono_to_color(
    X: np.ndarray, mean=None, std=None,
    norm_max=None, norm_min=None, eps=1e-6
):
    # Stack X as [X,X,X]
    X = np.stack([X, X, X], axis=-1)

    # Standardize
    mean = mean or X.mean()
    X = X - mean
    std = std or X.std()
    Xstd = X / (std + eps)
    _min, _max = Xstd.min(), Xstd.max()
    norm_max = norm_max or _max
    norm_min = norm_min or _min
    if (_max - _min) > eps:
        # Normalize to [0, 255]
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V

In [8]:
def get_criterion():
    pos_weights = torch.ones(SPECIES_NUM)
    pos_weights = pos_weights * SPECIES_NUM
    loss_function = nn.BCEWithLogitsLoss(pos_weight=pos_weights)
    return loss_function

In [9]:
class LitModule(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.encoder = torch.hub.load('zhanghang1989/ResNeSt', 'resnest50', pretrained=True)
        self.encoder.fc = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(1024, SPECIES_NUM)
        )
        
        self.criterion = get_criterion()
        
    def forward(self, x):
        x_out = self.encoder(x)
        return x_out
    
    def configure_optimizers(self):
        optimizer = torch.optim.SGD(model.parameters(), lr=0.01, weight_decay=0.0001, momentum=0.9)
        return optimizer
    
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        y_pred = self.encoder(x)    
        loss = self.criterion(y_pred, y)
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        y_pred = self.encoder(x)
        loss = self.criterion(y_pred, y)
        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    
    def validation_epoch_end(self, validation_step_outputs):
        mean_loss = torch.stack([x for x in validation_step_outputs]).mean()
        print('valid_epoch_loss = ', mean_loss)
        self.log('valid_epoch_loss', mean_loss, prog_bar=True, logger=True)
#         tqdm.write('Dice: \t%.3f' % mean_loss)
        return mean_loss

In [10]:
def signal_to_mel(y, sr, mel_params):
    
    len_y = len(y)
    effective_length = int(SR * PERIOD)
    
    start = 0
    end = start + effective_length
    
    images = []
    while(start < len_y):
        if(end > len_y):
            break
        y_ele = y[start:end]
        
        melspec = librosa.feature.melspectrogram(y_ele, sr=sr, **mel_params['melspectrogram_parameters'])
        melspec = librosa.power_to_db(melspec).astype(np.float32)
    
        image = mono_to_color(melspec)
        height, width, _ = image.shape
        image = cv2.resize(image, (int(width * mel_params['img_size'] / height), mel_params['img_size']))
        image = np.moveaxis(image, 2, 0)
        image = (image / 255.0).astype(np.float32)
#         image = torch.from_numpy(image).clone()
        images.append(image)
        
        start = end
        end += effective_length
        
    return np.array(images)

In [11]:
test_wav_pathes = sorted(glob.glob(str(TEST_AUDIO_DIR / '*.flac')))
len(test_wav_pathes)

1992

In [12]:
device = torch.device("cuda")

In [13]:
model_pathes = sorted(glob.glob('./output/model*'))
model_pathes

['./output/model0',
 './output/model1',
 './output/model2',
 './output/model3',
 './output/model4']

In [14]:
for i, model_path in enumerate(model_pathes):
    model = LitModule()
    model.load_state_dict(torch.load(model_path))
    model.eval().to(device)
    preds = []

    for path in tqdm(test_wav_pathes):
        y, sr = sf.read(path)
        mel_img = signal_to_mel(y, sr, config["dataset"]["params"])
        mel_img = torch.from_numpy(mel_img).clone().to(device)
        pred = model(mel_img)
        pred = nn.Softmax()(pred)
        pred = torch.mean(pred, 0)
        pred = pred.to('cpu').detach().numpy().copy()
        preds.append(pred)

    preds = np.array(preds)
    sub = pd.read_csv(str(RAW_DATA / 'sample_submission.csv'))
    sub.loc[:, 's0':'s23'] = preds
    sub.to_csv(OUTPUT_DIR + '37sub' + str(i) + '.csv', index=False)
    #     break

Using cache found in /home/user/.cache/torch/hub/zhanghang1989_ResNeSt_master
  0%|          | 0/1992 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  if sys.path[0] == '':
100%|██████████| 1992/1992 [17:39<00:00,  1.88it/s]
Using cache found in /home/user/.cache/torch/hub/zhanghang1989_ResNeSt_master
100%|██████████| 1992/1992 [17:49<00:00,  1.86it/s]
Using cache found in /home/user/.cache/torch/hub/zhanghang1989_ResNeSt_master
100%|██████████| 1992/1992 [17:42<00:00,  1.87it/s]
Using cache found in /home/user/.cache/torch/hub/zhanghang1989_ResNeSt_master
100%|██████████| 1992/1992 [17:49<00:00,  1.86it/s]
Using cache found in /home/user/.cache/torch/hub/zhanghang1989_ResNeSt_master
100%|██████████| 1992/1992 [17:39<00:00,  1.88it/s]
